In [1]:
import csv 
import requests
import wikipediaapi
import classes as cs
import json
import networkx as nx
import graph_networkx as gnx
import nx_altair as nxa
import pandas as pd
import altair as alt
import network_functions as ntf 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Testing file: so that I don't have to contstantly rerun

In [2]:
#graph = gnx.createMyGraph()
graph = ntf.load_example_graph()

#print(graph.nodes)
#returns info --  print(mygraph.nodes['Katy Perry'])


In [3]:
print(graph.nodes['Jensen McRae'])

{'api_link': 'https://api.spotify.com/v1/artists/11dABkjSoOjcP9p3TFSNRj', 'genres': ['gen z singer-songwriter'], 'img_info': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebd4290c7602907221e584aa11', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174d4290c7602907221e584aa11', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178d4290c7602907221e584aa11', 'width': 160}], 'popularity': 42, 'birth': 'September 10, 1997 Santa Monica, California', 'died': 'alive', 'instruments': ['vocals,', 'piano,', 'guitar'], 'occupations': ['singer-songwriter,', 'poet'], 'type': 'artist'}


In [ ]:

color_map = []
for node in graph:
    values = graph.nodes[node]
    if values['type'] == 'artist':
        color_map.append('red') #artist
    else: 
        color_map.append('pink') #genre

nx.draw_networkx(graph,with_labels=False,node_color=color_map,node_size=20) #.interactive()

In [6]:
for n in graph.nodes():
        graph.nodes[n]['id']=str(n)

def draw_network_old(graph):
        return(nxa.draw_networkx(graph).encode(tooltip=['id:N'],fill='type:N').properties(width=500,height=500,title='Top 100 Playlist Graph').interactive())


ntf.draw_network(graph,False,100)

alt.LayerChart(...)

In [ ]:
def draw_network(graph,labels=True,size_v=300):        #labels is true or false
    position = nx.spring_layout(graph)

    nodelayer = nxa.draw_networkx_nodes(graph,pos=position)
    edgeslayer = nxa.draw_networkx_edges(graph,pos=position)
    n1=nodelayer.mark_circle(size=size_v,opacity=1).encode(color='type:N',tooltip=['id:N'])
    text = nodelayer.mark_text(color='black').encode(
        text='id:N'
        )
    if labels:
        return(edgeslayer+n1+text)
    return((edgeslayer+n1).interactive())
#draw_network(graph,False,100).properties(width=500,height=500)



In [ ]:
graph.nodes['melancholia']

In [ ]:
nx.shortest_path(graph,source='uk pop',target='vancouver indie')

In [ ]:
#here I want to get all artists & genres
def get_genre_artists(graph):
    '''
    Gets easy to parse lists of all the artists in a graph and all the genres.

    Returns the lists in a dict with keys 'artists' and 'genres'

    PARAMETERS
    ----------
    graph: networkx graph
        graph with nodes and edges, where every node has attributes id and type
    
    RETURNS
    -------
    dict: key string, value list
        dict with keys 'artists' & 'genres' with lists of all artists & genres respectively
    
    '''
    artist_list = []
    genre_list = []

    for n in graph.nodes():
        if graph.nodes[n]['type']=='artist':
            artist_list.append(n)
        elif graph.nodes[n]['type']=='genre':
            genre_list.append(n)
        else:
            print(n)
    return({'artists':artist_list,'genres':genre_list})

get_genre_artists(graph)

In [ ]:
#def filter_edge
def get_focus_graph(graph,name):
        neighbors = [n for n in graph.neighbors(name)]

        def filter_node(n1):
            if n1 ==name: 
                return True
            return (n1 in neighbors)
        
        sub_graph = nx.subgraph_view(graph,filter_node)
        return sub_graph


flor = get_focus_graph(graph,'flor')
draw_network(flor,labels=True,size_v=1000)

In [ ]:
#nx.shortest_path(graph,source='Daft Punk',target='Daddy Yankee')
def get_shortest_graph(graph,name,end):
        try:
            neighbors = nx.shortest_path(graph,source=name,target=end) #THIS THROWS THE ERROR

            def filter_node(n1):
                if n1 ==name: 
                    return True
                return (n1 in neighbors)
            
            sub_graph = nx.subgraph_view(graph,filter_node)
            return(sub_graph)
        except:
            return None
        # position = nx.kamada_kawai_layout(sub_graph)

        # nodelayer = nxa.draw_networkx_nodes(sub_graph,pos=position)
        # edgeslayer = nxa.draw_networkx_edges(sub_graph,pos=position)
        # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
        # text = nodelayer.mark_text(color='black').encode(
        #     text='id'
        #     ).properties(title=f"From {name} to {end}")
        # return(edgeslayer+n1+text)

short = get_shortest_graph(graph,'uk pop','vancouver indie') 
draw_network(short,labels=True,size_v=1000)

In [ ]:
#print(graph.nodes['Daft Punk'])

In [ ]:
def search_dead(graph): 
    def filter_node(n1):
            if graph.nodes[n1]['type']=='genre': #if its a genre its not dead
                return False
            elif graph.nodes[n1]['died']== None:
                 return False
            elif graph.nodes[n1]['died']=='alive':
                 return False
            return True
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        #similar helper function to one above, returns true or false if the node should be included
        #filters to include the nodes from sub_graph and their neighbors
        if n1 in list(sub_graph.nodes): 
            return True
        return (n1 in nbrs)

    return nx.subgraph_view(graph,filter_genres)


    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N' 
    #     ).properties(title=f"Dead artists")
    # return(n1+text) ###SKYELER MAKE A CHANGE-- IF NO NODES, RETURN NONE


dead=search_dead(graph)
draw_network(dead,labels=True,size_v=1000)

In [7]:
from networkx.readwrite import json_graph
print(json_graph.node_link_data(graph))

thing = json_graph.node_link_data(graph)

#dumped_json_cache = json.dumps(cache_dict)
# fw = open('test_graph_data',"w")
# dumped_json_cache = json.dumps(thing)
# fw.write(dumped_json_cache)
# fw.close() 

def load_example_graph(filename='test_graph_data'):
    ex_file = open(filename,"r")
    graph_contents = ex_file.read()
    graph_dict = json.loads(graph_contents)
    ex_file.close()
    return nx.node_link_graph(graph_dict)

#print(load_example_graph())


{'directed': False, 'multigraph': False, 'graph': {}, 'nodes': [{'api_link': 'https://api.spotify.com/v1/artists/7N15xabCPo5XjsUSTnGVes', 'genres': [], 'img_info': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebfae2e5d2180bebe2210df4aa', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174fae2e5d2180bebe2210df4aa', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178fae2e5d2180bebe2210df4aa', 'width': 160}], 'popularity': 22, 'birth': None, 'died': None, 'instruments': [], 'occupations': [], 'type': 'artist', 'id': 'Lia Pappas-Kemps'}, {'api_link': 'https://api.spotify.com/v1/artists/11dABkjSoOjcP9p3TFSNRj', 'genres': ['gen z singer-songwriter'], 'img_info': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebd4290c7602907221e584aa11', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174d4290c7602907221e584aa11', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/

In [ ]:
def search_alive(graph): #MAYBE ADD LIVING AND DEAD GENRES TOOO???
    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre': #if its a genre its not dead
            return False
        if graph.nodes[n1]['died'] is None:
             return False
        elif graph.nodes[n1]['died']=='alive':
                return True
        return False
    sub_graph= nx.subgraph_view(graph,filter_node)
    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        #similar helper function to one above, returns true or false if the node should be included
        #filters to include the nodes from sub_graph and their neighbors
        if n1 in list(sub_graph.nodes): 
            return True
        return (n1 in nbrs)

    return nx.subgraph_view(graph,filter_genres)
    # position = nx.kamada_kawai_layout(sub_graph)

    # nodelayer = nxa.draw_networkx_nodes(sub_graph,pos=position)
    # #edgeslayer = nxa.draw_networkx_edges(sub_graph,pos=position)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N',tooltip=['id:N'])
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id'
    #     ).properties(title=f"Alive artists")
    # return(n1+text)

alive = search_alive(graph)
draw_network(alive,labels=True,size_v=200)

In [ ]:
def search_from(graph,place): ###DOUBLES AS YEAR OF BIRTH

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if graph.nodes[n1]['birth']==None:
            return False
        elif place.lower() in graph.nodes[n1]['birth'].lower(): #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        nbrs += [n for n in graph.neighbors(node)]

    def filter_genres(n1):
        if n1 in list(sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    

    return nx.subgraph_view(graph,filter_genres)


    # position = nx.kamada_kawai_layout(and_genres)
    # #SKYELER -- could onsider adding some cool cheeky tooltips

    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position).properties(width=500,height=500)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position).properties(width=500,height=500)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N'
    #     ).properties(title=f"Artists from {place}")
    # return(edgeslayer+n1+text)

england = search_from(graph,'England')
old = search_from(graph,'1997')
draw_network(old,labels=True,size_v=200)

In [ ]:
def search_occupations(graph,job): 

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if graph.nodes[n1]['occupations']==None:
            return False
        elif job.lower() in graph.nodes[n1]['occupations']: #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in list(sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    
    

    return nx.subgraph_view(graph,filter_genres)


    # position = nx.kamada_kawai_layout(and_genres)

    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position).properties(height=500,width=500)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position).properties(height=500,width=500)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N'
    #     ).properties(title=f"Artists with a {job} career")
    # return(edgeslayer+n1+text)

poem_girls = search_occupations(graph,'poet')
draw_network(old,labels=True,size_v=200)

In [ ]:
def search_instruments(graph,instru): 

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if graph.nodes[n1]['instruments']==None:
            return False
        elif instru.lower() in graph.nodes[n1]['instruments']: #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in list(sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    
    return nx.subgraph_view(graph,filter_genres)


    # position = nx.kamada_kawai_layout(and_genres)
    # #SKYELER -- could onsider adding some cool cheeky tooltips

    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N'
    #     ).properties(title=f"Who can play the {instru}?") ##Make this dynamic for vocals!
    return(edgeslayer+n1+text)

vocals = search_instruments(graph,'vocals')
draw_network(vocals,labels=True,size_v=200)

In [ ]:
def calculate_avg_popularity(graph):
    count = 0
    sum = 0

    for n in graph.nodes:
        if graph.nodes[n]['type']!='genre':
            sum +=graph.nodes[n]['popularity']
            count+=1
    return (sum/count)

print(calculate_avg_popularity(graph))

In [18]:
def filter_popularity(graph,pop): #greater than

    def filter_node(n1):
        if graph.nodes[n1]['type']=='genre':
            return False
        if graph.nodes[n1]['popularity']==None:
            return False
        elif int(graph.nodes[n1]['popularity'])>=int(pop): #if its a genre its not dead
            return True
        return False
    
    sub_graph = nx.subgraph_view(graph,filter_node)

    nbrs = []

    for node in sub_graph:
        neighbors = [n for n in graph.neighbors(node)]
        nbrs += neighbors #could clean this up

    def filter_genres(n1):
        if n1 in list(sub_graph.nodes): 
            return True
        elif graph.nodes[n1]['type']!='genre':
            return False
        return (n1 in nbrs)
    

    return nx.subgraph_view(graph,filter_genres)


    # position = nx.kamada_kawai_layout(and_genres)

    # nodelayer = nxa.draw_networkx_nodes(and_genres,pos=position)
    # edgeslayer = nxa.draw_networkx_edges(and_genres,pos=position)
    # n1=nodelayer.mark_circle(size=1000,opacity=1).encode(color='type:N')
    # text = nodelayer.mark_text(color='black').encode(
    #     text='id:N'
    #     ).properties(title=f"Artists with a popularity score greater than {pop}") ##Make this dynamic for vocals!
    # return(edgeslayer+n1+text)

poppy = filter_popularity(graph,'60')
#ntf.draw_network(poppy,labels=True,size_v=400).interactive()
print(json_graph.node_link_data(poppy))

{'directed': False, 'multigraph': False, 'graph': {}, 'nodes': [{'api_link': 'https://api.spotify.com/v1/artists/2RVvqRBon9NgaGXKfywDSs', 'genres': ['alt z', 'uk pop'], 'img_info': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb8b521134ae0ba3f60aab6811', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab676161000051748b521134ae0ba3f60aab6811', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f1788b521134ae0ba3f60aab6811', 'width': 160}], 'popularity': 64, 'birth': 'Maisie Hannah Peters 28 May 2000 Steyning, England', 'died': 'alive', 'instruments': ['vocals', 'guitar'], 'occupations': ['singer-songwriter'], 'type': 'artist', 'id': 'Maisie Peters'}, {'type': 'genre', 'id': 'alt z'}, {'type': 'genre', 'id': 'uk pop'}, {'api_link': 'https://api.spotify.com/v1/artists/7GlBOeep6PqTfFi59PTUUN', 'genres': ['indie pop', 'springfield mo indie'], 'img_info': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebcde5a0d57c1b79d

In [11]:
#Let's get your genres 
def top_genres_bar(graph):
    genres = []

    for gnode in graph.nodes:
        if graph.nodes[gnode]['type']=='genre':
            genres.append([gnode,len([n for n in graph.neighbors(gnode)])])

    genre_popularity = pd.DataFrame(genres,columns = ['Genres',"Count of Artists"])
    #genre_popularity

    return alt.Chart(genre_popularity).mark_bar().encode(
        x=alt.X('Count of Artists:Q'),
        y=alt.Y('Genres:N',
                sort='-x')).transform_window(
        rank='rank(Neighbors)',
        #sort=[alt.SortField('Neighbors', order='descending')]
    ).transform_filter(
        (alt.datum.rank < 10)
    ).properties(
        title='Your top genres'
    )

top_genres_bar(graph)

alt.Chart(...)

In [10]:
import pprint
pprint.pprint({"directed": False, "multigraph": False, "graph": {}, "nodes": [{"api_link": "https://api.spotify.com/v1/artists/7N15xabCPo5XjsUSTnGVes", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebfae2e5d2180bebe2210df4aa", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174fae2e5d2180bebe2210df4aa", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178fae2e5d2180bebe2210df4aa", "width": 160}], "popularity": 22, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Lia Pappas-Kemps"}, {"api_link": "https://api.spotify.com/v1/artists/11dABkjSoOjcP9p3TFSNRj", "genres": ["gen z singer-songwriter"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebd4290c7602907221e584aa11", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174d4290c7602907221e584aa11", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178d4290c7602907221e584aa11", "width": 160}], "popularity": 42, "birth": "September 10, 1997 Santa Monica, California", "died": "alive", "instruments": ["vocals,", "piano,", "guitar"], "occupations": ["singer-songwriter,", "poet"], "type": "artist", "id": "Jensen McRae"}, {"type": "genre", "id": "gen z singer-songwriter"}, {"api_link": "https://api.spotify.com/v1/artists/2RVvqRBon9NgaGXKfywDSs", "genres": ["alt z", "uk pop"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb8b521134ae0ba3f60aab6811", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051748b521134ae0ba3f60aab6811", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1788b521134ae0ba3f60aab6811", "width": 160}], "popularity": 64, "birth": "Maisie Hannah Peters 28 May 2000 Steyning, England", "died": "alive", "instruments": ["vocals", "guitar"], "occupations": ["singer-songwriter"], "type": "artist", "id": "Maisie Peters"}, {"type": "genre", "id": "alt z"}, {"type": "genre", "id": "uk pop"}, {"api_link": "https://api.spotify.com/v1/artists/1strrexNHv8TNVfVnMRhVD", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebc418a840ca7d9a160ff92e2b", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174c418a840ca7d9a160ff92e2b", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178c418a840ca7d9a160ff92e2b", "width": 160}], "popularity": 19, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "WATERBEAR"}, {"api_link": "https://api.spotify.com/v1/artists/2wCyYQ7ZZm89ylc7Ryryyl", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb14c0cc9365fd4b364fd58795", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517414c0cc9365fd4b364fd58795", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17814c0cc9365fd4b364fd58795", "width": 160}], "popularity": 21, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Celia"}, {"api_link": "https://api.spotify.com/v1/artists/6RhkgeqhRai3jy4ULSlxFx", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb68dcf75016755b7c6ef4659a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517468dcf75016755b7c6ef4659a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17868dcf75016755b7c6ef4659a", "width": 160}], "popularity": 29, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Juliana Madrid"}, {"api_link": "https://api.spotify.com/v1/artists/7uXyXCNAbNu5X6BdPFBWrj", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb575e76a6a60ad3486154243", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b575e76a6a60ad3486154243", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b575e76a6a60ad3486154243", "width": 160}], "popularity": 18, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Chloe Dadd"}, {"api_link": "https://api.spotify.com/v1/artists/2HCsMc18MHvpDACmNhbpQ5", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb9b275786d57f0d3843ec7c3a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051749b275786d57f0d3843ec7c3a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1789b275786d57f0d3843ec7c3a", "width": 160}], "popularity": 20, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Lauren Juzang"}, {"api_link": "https://api.spotify.com/v1/artists/73SW9LnBmuff938R5ZbqK4", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb0dee0233476ad03554583759", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051740dee0233476ad03554583759", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1780dee0233476ad03554583759", "width": 160}], "popularity": 40, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "ok.danke.tschüss"}, {"api_link": "https://api.spotify.com/v1/artists/31431J9PD3bfNsPKkezt0d", "genres": ["alt z"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebda1806771c224061133b4ba3", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174da1806771c224061133b4ba3", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178da1806771c224061133b4ba3", "width": 160}], "popularity": 47, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Leanna Firestone"}, {"api_link": "https://api.spotify.com/v1/artists/12dfSc1rVtEea2qMe38v3T", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebf5453e0b737411798adc2588", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174f5453e0b737411798adc2588", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178f5453e0b737411798adc2588", "width": 160}], "popularity": 20, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Kayla Grace"}, {"api_link": "https://api.spotify.com/v1/artists/6Lk699bosWcOqCMFLJFrPp", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb3d60b6477e8fea612006a596", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051743d60b6477e8fea612006a596", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1783d60b6477e8fea612006a596", "width": 160}], "popularity": 34, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "MELE"}, {"api_link": "https://api.spotify.com/v1/artists/6ws5XBA70XgeBpnLZhQBoy", "genres": ["toronto indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb0266f599dc5bc8dd1f854528", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051740266f599dc5bc8dd1f854528", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1780266f599dc5bc8dd1f854528", "width": 160}], "popularity": 55, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "The Beaches"}, {"type": "genre", "id": "toronto indie"}, {"api_link": "https://api.spotify.com/v1/artists/4WnfD0e5J91W7zyHBK6svA", "genres": ["german indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb4398cf15db77745024c9791", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b4398cf15db77745024c9791", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b4398cf15db77745024c9791", "width": 160}], "popularity": 41, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Blond"}, {"type": "genre", "id": "german indie"}, {"api_link": "https://api.spotify.com/v1/artists/5oFIcDSdVrn8T3Ldes4Ebn", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebe2d0c06b794d6a89f185498f", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174e2d0c06b794d6a89f185498f", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178e2d0c06b794d6a89f185498f", "width": 160}], "popularity": 24, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Power Plush"}, {"api_link": "https://api.spotify.com/v1/artists/17l4XlVVWNktDeJDigQ3HJ", "genres": ["neue neue deutsche welle"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebcd34cd91c4d7f2d74e3bce61", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174cd34cd91c4d7f2d74e3bce61", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178cd34cd91c4d7f2d74e3bce61", "width": 160}], "popularity": 47, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Dilla"}, {"type": "genre", "id": "neue neue deutsche welle"}, {"api_link": "https://api.spotify.com/v1/artists/09Wl9YiRr5l1rChWktQD4o", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb7335a43c0879d46bc3d2162", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b7335a43c0879d46bc3d2162", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b7335a43c0879d46bc3d2162", "width": 160}], "popularity": 32, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "NoSo"}, {"api_link": "https://api.spotify.com/v1/artists/7GlBOeep6PqTfFi59PTUUN", "genres": ["indie pop", "springfield mo indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebcde5a0d57c1b79de5fce6bee", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174cde5a0d57c1b79de5fce6bee", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178cde5a0d57c1b79de5fce6bee", "width": 160}], "popularity": 60, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Chappell Roan"}, {"type": "genre", "id": "indie pop"}, {"type": "genre", "id": "springfield mo indie"}, {"api_link": "https://api.spotify.com/v1/artists/7E2aQQjErJocovYFjYLzWU", "genres": ["alt z", "singer-songwriter pop"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebc4901ea3e71577a765e8ff78", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174c4901ea3e71577a765e8ff78", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178c4901ea3e71577a765e8ff78", "width": 160}], "popularity": 55, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Emei"}, {"type": "genre", "id": "singer-songwriter pop"}, {"api_link": "https://api.spotify.com/v1/artists/3jCEVKIIkR6ayqeBhPLVxN", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebead3451e30760bde18565cfc", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174ead3451e30760bde18565cfc", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178ead3451e30760bde18565cfc", "width": 160}], "popularity": 21, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Moira & Claire"}, {"api_link": "https://api.spotify.com/v1/artists/6uwvfnsp74AHafIT1vxAG3", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebcc062eca0f23e75cac9cb717", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174cc062eca0f23e75cac9cb717", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178cc062eca0f23e75cac9cb717", "width": 160}], "popularity": 37, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "SOMOH"}, {"api_link": "https://api.spotify.com/v1/artists/6LfA72Ad6LWcDYOwUMqpce", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb2c8e4d080e4cb4ebba268a74", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051742c8e4d080e4cb4ebba268a74", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1782c8e4d080e4cb4ebba268a74", "width": 160}], "popularity": 17, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Casey Bishop"}, {"api_link": "https://api.spotify.com/v1/artists/0higlnOzPAny6nOraITrsP", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb8f11bbd8c08d7f7978596f6", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b8f11bbd8c08d7f7978596f6", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b8f11bbd8c08d7f7978596f6", "width": 160}], "popularity": 23, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Straats"}, {"api_link": "https://api.spotify.com/v1/artists/68HDLnp9oeBFmkT6trhkza", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb36626bd471c0f8d0f9cf8703", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517436626bd471c0f8d0f9cf8703", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17836626bd471c0f8d0f9cf8703", "width": 160}], "popularity": 33, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Ethan Hibbs"}, {"api_link": "https://api.spotify.com/v1/artists/6VgPyGeGO86DztjK7GCYT3", "genres": ["indie pop"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebc90be9494c9e147d0582429b", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174c90be9494c9e147d0582429b", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178c90be9494c9e147d0582429b", "width": 160}], "popularity": 42, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Boyish"}, {"api_link": "https://api.spotify.com/v1/artists/5vx4pDmiFDyKMhuOIgpiRv", "genres": ["gen z singer-songwriter"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb58cf0937223e435e34e29afb", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517458cf0937223e435e34e29afb", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17858cf0937223e435e34e29afb", "width": 160}], "popularity": 45, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Sydney Rose"}, {"api_link": "https://api.spotify.com/v1/artists/3P4vW5tzQvmuoNaFQqzy9q", "genres": ["alt z", "bedroom pop", "indie pop", "pov: indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb439e58a8522b25d500442663", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174439e58a8522b25d500442663", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178439e58a8522b25d500442663", "width": 160}], "popularity": 55, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "chloe moriondo"}, {"type": "genre", "id": "bedroom pop"}, {"type": "genre", "id": "pov: indie"}, {"api_link": "https://api.spotify.com/v1/artists/5yM4OHXlRyyq28o76xejrk", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb9a0f8197dd3c28f6ff4d97f6", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051749a0f8197dd3c28f6ff4d97f6", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1789a0f8197dd3c28f6ff4d97f6", "width": 160}], "popularity": 21, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Sophie Truax"}, {"api_link": "https://api.spotify.com/v1/artists/3zBlhaQIMQtlpUEwYQoOoM", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb450b55f6626580342969ce32", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174450b55f6626580342969ce32", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178450b55f6626580342969ce32", "width": 160}], "popularity": 22, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "frown line"}, {"api_link": "https://api.spotify.com/v1/artists/46QrQRXLvHhE54yH8J1fX7", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb0caf041b8811c7d395b5200f", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051740caf041b8811c7d395b5200f", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1780caf041b8811c7d395b5200f", "width": 160}], "popularity": 30, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Vines"}, {"api_link": "https://api.spotify.com/v1/artists/79SufB5T3sbrw9lxq2YrB8", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb00b41ed0e6a2d23002d5f395", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517400b41ed0e6a2d23002d5f395", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17800b41ed0e6a2d23002d5f395", "width": 160}], "popularity": 22, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Ahli"}, {"api_link": "https://api.spotify.com/v1/artists/0qMLMrYSgSJBquDGCOMERs", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb4dabb16c284861ba9976cce", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b4dabb16c284861ba9976cce", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b4dabb16c284861ba9976cce", "width": 160}], "popularity": 15, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Caity Krone"}, {"api_link": "https://api.spotify.com/v1/artists/3Z9pPrfELSOEIpIGqusQ3w", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb5c4f6b3156c5f9b41cd0fb5e", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051745c4f6b3156c5f9b41cd0fb5e", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1785c4f6b3156c5f9b41cd0fb5e", "width": 160}], "popularity": 19, "birth": "19 January 2002 Worthing, United Kingdom", "died": "alive", "instruments": [], "occupations": [], "type": "artist", "id": "Amy Lawton"}, {"api_link": "https://api.spotify.com/v1/artists/72TRHiF9vPzDe78D4PmHWv", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebebed1e68a65d595d2dbfea7c", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174ebed1e68a65d595d2dbfea7c", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178ebed1e68a65d595d2dbfea7c", "width": 160}], "popularity": 40, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Stevie Bill"}, {"api_link": "https://api.spotify.com/v1/artists/3JFpiBTgX5C0aIzePDlgXA", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb2384a8da2b341f5f12a32603", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051742384a8da2b341f5f12a32603", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1782384a8da2b341f5f12a32603", "width": 160}], "popularity": 39, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "SKYLAR"}, {"api_link": "https://api.spotify.com/v1/artists/694IAMr4pVMGMVOVn0MM0R", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb45dcd5544aa7d884a8131e79", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517445dcd5544aa7d884a8131e79", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17845dcd5544aa7d884a8131e79", "width": 160}], "popularity": 21, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Eliza Harrison Smith"}, {"api_link": "https://api.spotify.com/v1/artists/1MTlhWSyxv11aeqREGEeb3", "genres": ["socal indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb49d60506b9cc1fe8a0b99e40", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517449d60506b9cc1fe8a0b99e40", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17849d60506b9cc1fe8a0b99e40", "width": 160}], "popularity": 40, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Chase Petra"}, {"type": "genre", "id": "socal indie"}, {"api_link": "https://api.spotify.com/v1/artists/1lYNHBChDDWC233o6ribbt", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb8c9d4923e5e6aee82b0fef02", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051748c9d4923e5e6aee82b0fef02", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1788c9d4923e5e6aee82b0fef02", "width": 160}], "popularity": 43, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "EASHA"}, {"api_link": "https://api.spotify.com/v1/artists/504bzeQSGBk6rSsfrPjjvw", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb02ce075ed62d3bb05205cd95", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517402ce075ed62d3bb05205cd95", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17802ce075ed62d3bb05205cd95", "width": 160}], "popularity": 31, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Mel Bryant & the Mercy Makers"}, {"api_link": "https://api.spotify.com/v1/artists/4DO0vJg9aCzoOgCIDEifEu", "genres": ["boston indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebe1d7d8172242a635eaa59a4b", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174e1d7d8172242a635eaa59a4b", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178e1d7d8172242a635eaa59a4b", "width": 160}], "popularity": 25, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Them Fantasies"}, {"type": "genre", "id": "boston indie"}, {"api_link": "https://api.spotify.com/v1/artists/6Y2m4AEOS9JFrsK2goyg7T", "genres": ["indie pop"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebe97167346edf1caad804d2f1", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174e97167346edf1caad804d2f1", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178e97167346edf1caad804d2f1", "width": 160}], "popularity": 42, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Babygirl"}, {"api_link": "https://api.spotify.com/v1/artists/2Sf3JoQvmbE3hi7hfwzofq", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebf3c40cdb27985388873e205a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174f3c40cdb27985388873e205a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178f3c40cdb27985388873e205a", "width": 160}], "popularity": 38, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "milk."}, {"api_link": "https://api.spotify.com/v1/artists/6BPf3YvvXG87cFZPT9WQgx", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebcdbc5d667925e242b8653a38", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174cdbc5d667925e242b8653a38", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178cdbc5d667925e242b8653a38", "width": 160}], "popularity": 28, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Annika Kilkenny"}, {"api_link": "https://api.spotify.com/v1/artists/48LeC49ZUI6oXfZrTA2QlL", "genres": ["phoenix indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eba73967f6ef6934aa234b79eb", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174a73967f6ef6934aa234b79eb", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178a73967f6ef6934aa234b79eb", "width": 160}], "popularity": 34, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Good Boy Daisy"}, {"type": "genre", "id": "phoenix indie"}, {"api_link": "https://api.spotify.com/v1/artists/0i4M8k5IcQpiEH6nBMdfPT", "genres": ["indie pop", "vancouver indie"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb07a4d0ce13953ef833ba9ff3", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517407a4d0ce13953ef833ba9ff3", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17807a4d0ce13953ef833ba9ff3", "width": 160}], "popularity": 39, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Haley Blais"}, {"type": "genre", "id": "vancouver indie"}, {"api_link": "https://api.spotify.com/v1/artists/6EEkobfnenJF603WO8nUrR", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebd200a127505bd2a1f316ec3a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174d200a127505bd2a1f316ec3a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178d200a127505bd2a1f316ec3a", "width": 160}], "popularity": 32, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Willow Avalon"}, {"api_link": "https://api.spotify.com/v1/artists/4wMfqR1EZagrSlYndItxGQ", "genres": ["small room"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb55a7e92a1cdfeb004388a932", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517455a7e92a1cdfeb004388a932", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17855a7e92a1cdfeb004388a932", "width": 160}], "popularity": 44, "birth": "May 2, 1991", "died": "alive", "instruments": [], "occupations": [], "type": "artist", "id": "Field Medic"}, {"type": "genre", "id": "small room"}, {"api_link": "https://api.spotify.com/v1/artists/0szWPxzzE8DVEfXFRCLBUb", "genres": ["hopebeat", "indie poptimism", "metropopolis"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb3572d3df91e9299cb978a683", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051743572d3df91e9299cb978a683", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1783572d3df91e9299cb978a683", "width": 160}], "popularity": 44, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "flor"}, {"type": "genre", "id": "hopebeat"}, {"type": "genre", "id": "indie poptimism"}, {"type": "genre", "id": "metropopolis"}, {"api_link": "https://api.spotify.com/v1/artists/5df6GUsn0QFDhOV30drxtq", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb08f04dfd57a310c031c5ed46", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517408f04dfd57a310c031c5ed46", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17808f04dfd57a310c031c5ed46", "width": 160}], "popularity": 14, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Alt Fiction"}, {"api_link": "https://api.spotify.com/v1/artists/3HljFuKgpaXNKfIlFZWCEk", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebb149a14cf9e3b1c1baaab19a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174b149a14cf9e3b1c1baaab19a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178b149a14cf9e3b1c1baaab19a", "width": 160}], "popularity": 30, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Jesse Detor"}, {"api_link": "https://api.spotify.com/v1/artists/7AZm56bjPk0tYl6LTyJu9N", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb4ff9d7bc085f621cbfbac66a", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051744ff9d7bc085f621cbfbac66a", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1784ff9d7bc085f621cbfbac66a", "width": 160}], "popularity": 27, "birth": None, "died": "alive", "instruments": [], "occupations": [], "type": "artist", "id": "Blusher"}, {"api_link": "https://api.spotify.com/v1/artists/1GERDglQrxe4ynLzcdG6qP", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb13997156c925c67fe1737c6e", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab6761610000517413997156c925c67fe1737c6e", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f17813997156c925c67fe1737c6e", "width": 160}], "popularity": 55, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Evan Honer"}, {"api_link": "https://api.spotify.com/v1/artists/7qrEXiLLnWkkYHhadZ1Oij", "genres": ["bubblegrunge"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5eb7b474068e03798fc87471df7", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab676161000051747b474068e03798fc87471df7", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f1787b474068e03798fc87471df7", "width": 160}], "popularity": 45, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Blondshell"}, {"type": "genre", "id": "bubblegrunge"}, {"api_link": "https://api.spotify.com/v1/artists/3z3RfAKT2rmzCfPoXSKfEA", "genres": ["alt z"], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebc20a8c793e51e0f4aede26ef", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174c20a8c793e51e0f4aede26ef", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178c20a8c793e51e0f4aede26ef", "width": 160}], "popularity": 23, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "Imani Graham"}, {"api_link": "https://api.spotify.com/v1/artists/10YC7gfEVGGez94KMAJoLx", "genres": [], "img_info": [{"height": 640, "url": "https://i.scdn.co/image/ab6761610000e5ebf140ffc55c0b8cfbe1220ba1", "width": 640}, {"height": 320, "url": "https://i.scdn.co/image/ab67616100005174f140ffc55c0b8cfbe1220ba1", "width": 320}, {"height": 160, "url": "https://i.scdn.co/image/ab6761610000f178f140ffc55c0b8cfbe1220ba1", "width": 160}], "popularity": 40, "birth": None, "died": None, "instruments": [], "occupations": [], "type": "artist", "id": "chlothegod"}], "links": [{"source": "Jensen McRae", "target": "gen z singer-songwriter"}, {"source": "gen z singer-songwriter", "target": "Sydney Rose"}, {"source": "Maisie Peters", "target": "alt z"}, {"source": "Maisie Peters", "target": "uk pop"}, {"source": "alt z", "target": "Leanna Firestone"}, {"source": "alt z", "target": "Emei"}, {"source": "alt z", "target": "chloe moriondo"}, {"source": "alt z", "target": "Imani Graham"}, {"source": "The Beaches", "target": "toronto indie"}, {"source": "Blond", "target": "german indie"}, {"source": "Dilla", "target": "neue neue deutsche welle"}, {"source": "Chappell Roan", "target": "indie pop"}, {"source": "Chappell Roan", "target": "springfield mo indie"}, {"source": "indie pop", "target": "Boyish"}, {"source": "indie pop", "target": "chloe moriondo"}, {"source": "indie pop", "target": "Babygirl"}, {"source": "indie pop", "target": "Haley Blais"}, {"source": "Emei", "target": "singer-songwriter pop"}, {"source": "chloe moriondo", "target": "bedroom pop"}, {"source": "chloe moriondo", "target": "pov: indie"}, {"source": "Chase Petra", "target": "socal indie"}, {"source": "Them Fantasies", "target": "boston indie"}, {"source": "Good Boy Daisy", "target": "phoenix indie"}, {"source": "Haley Blais", "target": "vancouver indie"}, {"source": "Field Medic", "target": "small room"}, {"source": "flor", "target": "hopebeat"}, {"source": "flor", "target": "indie poptimism"}, {"source": "flor", "target": "metropopolis"}, {"source": "Blondshell", "target": "bubblegrunge"}]})

{'directed': False,
 'graph': {},
 'links': [{'source': 'Jensen McRae', 'target': 'gen z singer-songwriter'},
           {'source': 'gen z singer-songwriter', 'target': 'Sydney Rose'},
           {'source': 'Maisie Peters', 'target': 'alt z'},
           {'source': 'Maisie Peters', 'target': 'uk pop'},
           {'source': 'alt z', 'target': 'Leanna Firestone'},
           {'source': 'alt z', 'target': 'Emei'},
           {'source': 'alt z', 'target': 'chloe moriondo'},
           {'source': 'alt z', 'target': 'Imani Graham'},
           {'source': 'The Beaches', 'target': 'toronto indie'},
           {'source': 'Blond', 'target': 'german indie'},
           {'source': 'Dilla', 'target': 'neue neue deutsche welle'},
           {'source': 'Chappell Roan', 'target': 'indie pop'},
           {'source': 'Chappell Roan', 'target': 'springfield mo indie'},
           {'source': 'indie pop', 'target': 'Boyish'},
           {'source': 'indie pop', 'target': 'chloe moriondo'},
           {'sou